# Baseline Model with Kaggle data: 2 epochs and 100x100 resolution

In [ ]:
import matplotlib.pyplot as plt
import pathlib, os, random
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import mlflow

#from modeling.config import EXPERIMENT_NAME
from config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential

## 1. Look at data

In [ ]:
# Get current absolute path of parent folder of this file
path_notebooks = os.path.dirname(os.path.abspath('1_baseline_kaggle_mlflow_test'))

# Get path of parent directory, i.e. path to repo
path = os.path.abspath(os.path.join(path_notebooks, os.pardir))

In [ ]:
DATASET = 'kaggle' # or 'kaggle+europe' or 'europe'

train_dir = path + '/data/data_2/train/'
test_dir = path + '/data/data_2/test/'
val_dir = path + '/data/data_2/valid/'

In [ ]:
data_dir = pathlib.Path(path + "/data/data_2/train/")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # creating a list of class names from subdirectory 
print(class_names)

In [ ]:
class_names = class_names[1:]
class_names

In [ ]:
def view_random_image(target_dir, target_class):
  # setting up the image directory
  target_folder = target_dir + target_class

  #get a random image path
  random_image = random.sample(os.listdir(target_folder), 1)

  #read image and plotting it
  img = mpimg.imread(target_folder + "/" + random_image[0] )
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off")

  print(f"Image shape: {img.shape}")
  
  return img

In [ ]:
img = view_random_image(target_dir = train_dir,
                        target_class = '/BAY-BREASTED WARBLER'
                        ) # or we can write nike instead of converse to see converse examples

In [ ]:
img.shape #(width, height, colour channels)

In [ ]:
plt.figure(figsize = (15,7))
plt.subplot(1,3,1)
steak_image = view_random_image(train_dir, "/BAR-TAILED GODWIT")
plt.subplot(1,3,2)
pizza_image = view_random_image(train_dir, "/YELLOW HEADED BLACKBIRD")
plt.subplot(1,3,3)
pizza_image = view_random_image(train_dir, "/INDIAN PITTA")

## 2. Data Preprocessing

In [ ]:
IMG_HEIGHT = 100
IMG_WIDTH = 100

In [ ]:
BATCH_SIZE = 32
# Rescale
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)

# data transfer from directories to batches
train_data = train_datagen.flow_from_directory(directory = train_dir,
                                               batch_size= BATCH_SIZE,
                                               target_size= (IMG_HEIGHT,IMG_WIDTH),
                                               class_mode = "categorical")

test_data = test_datagen.flow_from_directory(directory = test_dir,
                                               batch_size = BATCH_SIZE,
                                               target_size = (IMG_HEIGHT,IMG_WIDTH),
                                               class_mode = "categorical")

val_data = valid_datagen.flow_from_directory(directory = val_dir,
                                               batch_size = BATCH_SIZE,
                                               target_size = (IMG_HEIGHT,IMG_WIDTH),
                                               class_mode = "categorical")


## 3.Creating and fitting the Model

In [ ]:
# 1. Create a base model with tf.keras.applications
base_model = tf.keras.applications.InceptionV3(include_top= False,)

# 2. Freeze the base model
base_model.trainable = False

#3. Create inputs into models
inputs = tf.keras.layers.Input(shape =(IMG_HEIGHT,IMG_WIDTH,3), name = "input-layer")

#4. Rescaling
#x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

#5. Pass the inputs 
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

#7. Create the output activation layer
outputs = tf.keras.layers.Dense(450, activation = "softmax", name = "output-layer")(x)

In [ ]:
# 8. Combine the inputs with outputs into a model
LEARNING_RATE = 0.03
model_p100_e2 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_p100_e2.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE),
                metrics = ["accuracy"])


In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run(run_name='base model test')
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
EPOCHS = 2
history = model_p100_e2.fit(train_data,
                                 epochs=EPOCHS,
                                 steps_per_epoch = len(train_data),
                                 validation_data = val_data,
                                 validation_steps = int(0.25*len(val_data)),)

## Save model, history and parameters

Runtime info: One epoch with the res takes roughly 4-5 minutes - total runtime:27 minutes

In [ ]:
# Save the entire small model as a SavedModel.
#!mkdir -p saved_model

model_name = 'baseline_kaggle_p100_e2'
model_p100_e2.save(path + '/saved_model/' + model_name)

In [ ]:
# save history
with open(path + '/saved_model/' + model_name + '/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
param_dict = {}
param_dict['ds'] = DATASET
param_dict['lr'] = LEARNING_RATE
param_dict['bs'] = BATCH_SIZE
param_dict['eps'] = EPOCHS
param_dict['img_height'] = IMG_HEIGHT

# save model parameters
with open(path + '/saved_model/' + model_name + '/trainParamsDict', 'wb') as file_pi:
    pickle.dump(param_dict, file_pi)

In [ ]:
model_p100_e2.summary()

In [ ]:
model_p100_e2.evaluate(test_data)

In [ ]:
def plot_loss_curves(history):

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  #plot loss
  plt.plot(epochs, loss, label = "training_loss")
  plt.plot(epochs, val_loss, label = "val_loss")
  plt.title("loss")
  plt.xlabel("epochs")
  plt.legend()

  #plot accuracy
  plt.figure() 
  plt.plot(epochs, accuracy, label = "training_accuracy")
  plt.plot(epochs, val_accuracy, label = "val_accuracy")
  plt.title("accuracy")
  plt.xlabel("epochs")
  plt.legend()

In [ ]:
plot_loss_curves(history)

In [ ]:
#seting parameters that should be logged on MLFlow
#these parameters were used in feature engineering (inputing missing values)
#or parameters of the model (fit_intercept for Linear Regression model)
params = {
      "batch_size": BATCH_SIZE,
      "image_size": IMG_HEIGHT,
      "epochs": EPOCHS,
      "learning_rate": LEARNING_RATE,
  }

In [ ]:
#logging params to mlflow
mlflow.log_params(params)
#setting tags
mlflow.set_tag("running_from_jupyter", "True")
#logging metrics
mlflow.log_metric("train-" + "accuracy", history.history['accuracy'][-1])
mlflow.log_metric("valid-" + "accuracy", history.history['val_accuracy'][-1])
mlflow.log_metric("train-" + "loss", history.history['loss'][-1])
mlflow.log_metric("valid-" + "loss", history.history['val_loss'][-1])
# logging the model to mlflow will not work without a AWS Connection setup.. too complex for now
# but possible if running mlflow locally
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.keras.log_model(model_p100_e2 ,'InceptionV3')
mlflow.end_run()

In [ ]:
mlflow.get_run(run_id=run.info.run_id)

In [ ]:
!mlflow ui

## 4. Freeze top layers of Base Model

In [ ]:
# To begin fine-tuning lets start by setting the last 10 layers as trainable
base_model.trainable = True

# Un-freeze last 10 layers
for layer in base_model.layers[:-10]:
  layer.trainable = False

# Recompile (we have to compile model every time there is a change)
model_p100_e5.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), # when fine-tuning you typically want to lower lr by 10x
                 metrics = ["accuracy"] )


In [ ]:
# Check which layers are trainable
for layer_number, layer in enumerate(model_p100_e5.layers[1].layers):
  print(layer_number, layer.name, layer.trainable)

In [ ]:
# Now we have unfrozen some of the layers on the top 
print(len(model_p100_e5.trainable_variables))

## 5. Fine-Tuning and Refitting

In [ ]:
initial_epochs = 5
fine_tune_epochs = initial_epochs + 1

# Refit the model
history_2 = model_p100_e5.fit(train_data,
                       epochs = fine_tune_epochs,
                       validation_data = val_data,
                       validation_steps = int(0.25*len(val_data)),
                       initial_epoch =  history.epoch[-1],) # Start the epoch where it left before

In [ ]:
model_p100_e5.evaluate(test_data)

In [ ]:
plot_loss_curves(history_2)

In [ ]:
# Save the entire small model as a SavedModel.
!mkdir -p saved_model
model_p100_e5.save('saved_model/model_p100_e5')

## 6. Making some predictions

In [ ]:
# reminder: 
# img_height = 200
# img_width = 200

In [ ]:
# cReating a function that preprocess the custom data

def load_and_prep_image(filename, img_shape = img_height):
  img = tf.io.read_file(filename) #read image
  img = tf.image.decode_image(img) # decode the image to a tensor
  img = tf.image.resize(img, size = [img_height, img_width]) # resize the image
  img = img/255. # rescale the image
  return img

In [ ]:
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

Add new files to the test folder 

In [ ]:
amsel = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/Amsel.jpeg"
blaumeise = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/blaumeise.jpeg"
buchfink = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/buchfink.jpeg"
elster = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/elster.jpeg"
feldsperling = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/feldsperling.jpeg"
firefinch = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/firefinch.jpg"
gruenfink = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/grünfink.jpeg"
haussperling = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/haussperling.jpeg"
kohlmeise = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/Kohlmeise.jpeg"
mauersegler = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/mauersegler.jpeg"
mehlschwalbe = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/mehlschwalbe.jpeg"
rotkehlchen = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/Rotkehlchen.jpeg"
star = "/Users/friederikethies/neue_fische/ds-capstone-for-the-birds/data_2/images to test/Star.jpeg"

In [ ]:
pred_and_plot(model_p100_e5, amsel, class_names)

In [ ]:
history.history.keys()